# Equazioni differenziali del moto
## Spostamento di Studenti in un corridoio
<u> Simulazione uscita di scuola di un gruppo di alunni</u>

In [11]:
import pandas as pd
import numpy  as np
import math

In [12]:
tau_mean = 1
v_mean   = 1
#
# The number of student per classes is modeled by the mean and the variance of normal (Gaussian) distribution
p_mean, p_sigma = 10, 2.5

#
# The initial school configuration is modeled by multiply the class section for the number of age of courses for the random
# number of student per class
sezioni = ['A','B','C','D']
corso   = [1,2,3,4,5]
#
# The spatial features are calculated to rappresent all necessary
# room to contains the classes into unique floor

l       = len(sezioni)*len(corso)
pos    = {}
#lato aula
l_aula  = 5
#larghezza (dx) e lunghezza (dy) del corridoio
dx,dy   = 6,l_aula*(int(l/2)+1)
for i in range(int(l/2)+1):
    pos[2*i]={2*i:[l_aula,l_aula*i+2.5]}
    pos[2*i+1]={2*i+1:[l_aula+dx,l_aula*i+2.5]}

#Popolo le classi col numero di alunni
s       = np.random.normal(p_mean, p_sigma, l)
s       = s.astype('int')

#Associo gli alunni alle aule
aule    = {}
id      = 0
mtr     = 0
xsz,ysz = 0,0
for sz in sezioni:
    for c in corso:
        for i in range(s[id]):
            mtr = mtr+1
            aule[mtr]={'sezione':sz
            ,'anno':c
            ,'matricola':'MTR'+('00000'+str(mtr))[-5:]
            ,'x':pos[id][id][0]
            ,'y':pos[id][id][1]
            ,'tau':np.random.uniform(tau_mean/2,3*tau_mean/2)
            ,'V0':np.random.uniform(v_mean/2,3*v_mean/2)
}
        id = id+1

df_aule = pd.DataFrame.from_dict(aule,orient='index')
df_aule = df_aule.reset_index(inplace=False)


In [13]:
# Y axes model mouvement
def sy(s0,a0,v0,t0,T):
    '''
    spazio
    '''
    step=[]
    #a0=max(0,a0)
    #v0=max(0,v0)
    for t in range(0,T):
        if t<t0:
            step.extend([s0])
        else:
            ss=0.5*a0*(t-t0)*(t-t0)+v0*(t-t0)+s0
            step.extend([ss])
    return step

In [14]:
# X axes model mouvement
def sx(s0,xmin,xmax,tau,v0,t0,T):
    '''
    spazio
    '''
    step=[]
    xmax=xmax-0.5
    xmin=xmin+0.5
    for t in range(0,T):
        if t<t0:
            step.extend([s0])
        else:
            ss=s0+ v0*(t-t0)+v0*tau*(1-math.exp(-(t-t0)/tau))
            if ss>xmax:
                ss=xmax
            if ss<xmin:
                ss=xmin
            step.extend([ss])
    return step

In [15]:
T=5*60
S = pd.DataFrame()
for i in df_aule.index:
    #print(i)
    t0  = np.random.randint(60)
    teta= np.random.uniform(16*math.pi/40,19*math.pi/40)
    if df_aule.iloc[i]['x']>dx:
        p=pd.DataFrame([sx(s0=df_aule.iloc[i]['x']
            ,xmin=l_aula,xmax=l_aula+dx
            ,tau=df_aule.iloc[i]['tau'],v0=-df_aule.iloc[i]['V0']*math.cos(teta)
            ,t0=t0,T=T)
        ,sy(s0=df_aule.iloc[i]['y']
        ,a0=0,v0=df_aule.iloc[i]['V0']*math.sin(teta)
        ,t0=t0,T=T)]).T
    else:
        p=pd.DataFrame([sx(s0=df_aule.iloc[i]['x']
            ,xmin=l_aula,xmax=l_aula+dx
            ,tau=df_aule.iloc[i]['tau'],v0=df_aule.iloc[i]['V0']*math.cos(teta)
            ,t0=t0,T=T)
        ,sy(s0=df_aule.iloc[i]['y']
        ,a0=0,v0=df_aule.iloc[i]['V0']*math.sin(teta)
        ,t0=t0,T=T)]).T
    p.columns=['x','y']
    p['sezione']    = df_aule.iloc[i]['sezione']
    p['anno']       = df_aule.iloc[i]['anno']
    p['matricola']  = df_aule.iloc[i]['matricola']
    p['tau']         = df_aule.iloc[i]['tau']
    p['V0']         = df_aule.iloc[i]['V0']
    S=S.append(p)


In [16]:
S.to_csv('scenario_03_half.csv',sep=';')

In [17]:
S.index.name='t'
S=S.reset_index()

In [18]:
stat=S[S.x>l_aula]
stat=stat[stat.x<l_aula+dx]
stat=stat[stat.y<=60]
stat['S']=np.int32(stat['y'].values)
#stat.index.name='t'

In [19]:
stat=stat.groupby(by=['t','S']).count()

In [20]:
stat.to_csv('statistiche_03_half.csv',';')